In [ ]:
from google.colab import drive

drive.mount("gdrive")


Mounted at gdrive


In [ ]:
!pip install datasets -q
# Install all extra dependencies
!pip install bm25s[full] -q

# If you want to use stemming for better results, you can install a stemmer
!pip install PyStemmer -q

# To speed up the top-k selection process, you can install `jax`
!pip install jax[cpu] -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 29.0 MB/s eta 0:00:00


main idea bm25 + dpr + cross-embedder + reranker

In [ ]:
# хочется отдельно оценить bm25, dpr, bm25 + reranker, dpr + reranker


In [ ]:
from datasets import load_dataset

crypto_qa_hf_data = load_dataset("HemaChandrao/crypto_QA")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/44.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/16.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/180 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
corpus_data = []

for i in range(5):
    corpus_data.append(crypto_qa_hf_data["test"]["conversations"][i][1]["content"])

query = crypto_qa_hf_data["test"]["conversations"][0][0]["content"]


In [ ]:
print(query)

Every time I join a new crypto project Im worried about rug pulls What are the best practices to identify scams before investing


In [ ]:
print(corpus_data)

['Rug pulls Look out for anonymous teams no locked liquidity and toogoodtobetrue promises If its shady its probably a scam', "Oh no, a price bouncing off the 50-day MA?! ?? Don't worry, my friend, we can totes figure this out together! ?? First things first, let's take a deep breath and calm down those heart rate-racing emotions. ?? Next, let's check if there are any other indicators that could be signaling a potential reversal. Are there any other technical or fundamental factors at play? Maybe a recent news event or earnings announcement that could be causing the price movement? ?? Once we have a better understanding of what's going on, we can start thinking about how to adjust our trade strategy accordingly. After all, we don't want to get caught in a false hope trap! ?? So, what do you say? Are you ready to dive into this together and find the best course of action for your trade? ??", 'Minings energyintensive but green energy is the answer Solarpowered rigs anyone Bitcoins greener

# Metrics

In [ ]:
import numpy as np


def average_precision(pred_labels, true_labels):
    """
    Рассчитывает Average Precision (AP) для одного запроса.
    """
    num_relevant = np.sum(true_labels)
    if num_relevant == 0:
        return 0.0

    precision_sum = 0
    num_hits = 0

    for k in range(len(pred_labels)):
        if true_labels[pred_labels[k]] == 1:
            num_hits += 1
            precision_at_k = num_hits / (k + 1)
            precision_sum += precision_at_k

    return precision_sum / num_relevant

def mean_average_precision(predictions, ground_truths):
    """
    Рассчитывает Mean Average Precision (MAP).
    """
    aps = []
    for i in range(len(predictions)):
        ap = average_precision(predictions[i], ground_truths[i])
        aps.append(ap)

    return np.mean(aps)


# BM25

In [ ]:
import bm25s
import Stemmer


stemmer = Stemmer.Stemmer("english")
corpus_tokens = bm25s.tokenize(corpus_data, stopwords="en", stemmer=stemmer)

retriever = bm25s.BM25()
retriever.index(corpus_tokens)

query_tokens = bm25s.tokenize(query, stemmer=stemmer)

results, scores = retriever.retrieve(query_tokens, k=20)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")


Split strings:   0%|          | 0/5 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/5 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/5 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 2.20): 0
Rank 2 (score: 1.34): 1
Rank 3 (score: 0.67): 3
Rank 4 (score: 0.00): 2
Rank 5 (score: 0.00): 4


In [ ]:
import numpy as np

true_labels = np.array([
    1, 0, 0, 0, 0
]) # one-hot encoding
pred_labels = results[0]


mean_average_precision([pred_labels], [true_labels])

1.0

In [ ]:
import numpy as np

true_labels = np.array([
    0, 1, 0, 0, 0
])

In [ ]:
import numpy as np


def average_precision(pred_labels, true_labels):
    """
    Рассчитывает Average Precision (AP) для одного запроса.
    """
    num_relevant = np.sum(true_labels)
    if num_relevant == 0:  # Если нет релевантных элементов
        return 0.0

    precision_sum = 0
    num_hits = 0

    for k in range(len(pred_labels)):
        if true_labels[pred_labels[k]] == 1:  # Предсказание релевантно
            num_hits += 1
            precision_at_k = num_hits / (k + 1)  # Precision@k
            precision_sum += precision_at_k

    return precision_sum / num_relevant  # AP = средняя точность

def mean_average_precision(predictions, ground_truths):
    """
    Рассчитывает Mean Average Precision (MAP).
    """
    aps = []
    for i in range(len(predictions)):  # Предполагается, что несколько запросов
        ap = average_precision(predictions[i], ground_truths[i])
        aps.append(ap)

    return np.mean(aps)  # MAP = средний AP


MAP = mean_average_precision([pred_labels], [true_labels])
print(f"Mean Average Precision (MAP): {MAP:.4f}")


Mean Average Precision (MAP): 0.5000


In [ ]:
!pip install bm25s

In [ ]:
crypto_qa_hf_data["test"]["conversations"]


Every time I join a new crypto project Im worried about rug pulls What are the best practices to identify scams before investing


In [ ]:
import pandas as pd

retrieval_test_data = pd.read_csv(
    "/content/gdrive/MyDrive/aitalenthack/QnA-TEST(updated).csv"
)


In [ ]:
print(retrieval_test_data["title"].iloc[1])


The end of this year is approaching, how would you describe 2021 in terms of your crypto investments?


In [ ]:
print(retrieval_test_data["selftext"].iloc[1])


As the end of the year approaches, I am interested in how satisfied you are with 2021 in terms of crypto investment. \n\nI would say that another thing that is also very important in cryptocurrencies is the knowledge that could have been acquired this year.  \n\nAs every year, many ups and downs, new ATH, rising crypto market, many surprises, etc.  \n\nI would say that this year was very positive for the entire crypto market and that we are closing it with a great price for BTC, which knows to be much better. \n\n Are you satisfied? Are you in profit or loss? \n\nCheers!


In [ ]:
retrieval_test_data.columns

Index(['title', 'selftext', 'MAIN', 'comment_body', 'SyntheticID'], dtype='object')

In [ ]:
print(retrieval_test_data["MAIN"].iloc[0])


Which exchange to use to see holdings increase/decrease and PnL? Wazirx doesn't show how much a portfolio has changed (% change) and it is not possible to track how much a coin has changed. What do you guys do?


In [ ]:
print(retrieval_test_data["SyntheticID"].iloc[0])


QnA-1
